   mkdir bgee_coexpression  
   cd bgee_coexpression  
   wget ftp://ftp.bgee.org/current/download/calls/expr_calls/Homo_sapiens_expr_simple.tsv.zip  
   unzip Homo_sapiens_expr_simple.tsv.zip 

In [2]:
! ls -1

bar
bgee_coexp_network_gold.yaml
bgee_coexp_network.yaml
BGEE_coexpression_explore.ipynb
BGEE_coexpression_generate.ipynb
bgee_concordance.txt
bgee_gene_symbol_gold.txt
bgee_gene_symbol.txt
bgee_rank_degree.txt
ensgid_symbol_hgnc.yaml
ensg_ncbi.yaml
FA_1_core_complex.txt
FA_2_effector_proteins.txt
FA_3_associated_proteins.txt
foo
gene_gene_association.gv
gene_gene_association.gv~
gene_tissue_curve_first.yaml
gene_tissue_curve.yaml
gold_rank.txt
HGNC_nomenclature_update.ipynb
Homo_sapiens_expr_gold.tsv
Homo_sapiens_expr_simple.tsv
Homo_sapiens_expr_simple.tsv.zip
ncbi_symbol_hgnc.yaml
target_gene_symbol.txt
tissue_count_gold.png
tissue_count_gold.tab
tissue_count.png
tissue_count.tab
uberon.owl


: 1

In [3]:
head -1 Homo_sapiens_expr_simple.tsv | tr '\t' '\n' | grep -n .

1:Gene ID
2:"Gene name"
3:Anatomical entity ID
4:"Anatomical entity name"
5:Expression
6:Call quality
7:Expression rank


In [4]:
# we  have a three tier quality meteric
cut -f6 Homo_sapiens_expr_simple.tsv | sort | uniq -c | sort -n

      1 Call quality
 124978 bronze quality
2921827 gold quality
5837817 silver quality


Keeping all resulted in too many edges to be useful.  
redoing with just gold quality

In [7]:
grep 'present.*gold quality' Homo_sapiens_expr_simple.tsv > Homo_sapiens_expr_gold_present.tsv

In [8]:
# how many rows
wc -l < Homo_sapiens_expr_gold_present.tsv

1909771


In [9]:
# how many gene identifiers?
cut -f1 Homo_sapiens_expr_gold_present.tsv | sort -u | wc -l

55196


In [10]:
# how many gene symbols?
cut -f2 Homo_sapiens_expr_gold_present.tsv | sort -u | wc -l

53481


That is alot of gene ids & names,  
many more than _could_ have informative nomenclature  

note: not dealing with collapsing Ensembl id into "genes" here

In [11]:
# how many tissues?
cut -f3 Homo_sapiens_expr_gold_present.tsv | sort -u | wc -l

161


all is quality present & ansent is 312 tissues 
just gold quality is 217 tissues   
just gold and only present is 161 tissues 

#### for gold present it is  
55,196 * 161 == 8,886,556 possible  
                1,909,771 actual   

#### for gold it is  
59,154 * 217 = 12,836,418 possible  
               2,921,827 actual  
               
#### for all it is 

If every gene is checked for in every tissue  we will see  
59172 * 312 == 18,461,664  tissue-gene measurements
but we only see 8,884,623  rows   
so less than half gene-tissue combinations possible could be reported. 

In [12]:
cut -f3 Homo_sapiens_expr_gold_present.tsv | cut -f1 -d ':' |sort |uniq -c | sort -nr 

1880358 UBERON
  29413 CL


In [13]:
# tissue is ontology terms (yea!).
cut -f3 Homo_sapiens_expr_simple.tsv | cut -f1 -d ':' |sort |uniq -c | sort -nr 

8649326 UBERON
 235296 CL
      1 Anatomical entity ID


This makes for very interesting "future directions" where gene expression profiles
can be scored at various granularities based on anatomical windows, 
(if it turns out to be feasable)   
there are 312 tissue terms represented, 

In [15]:
cut -f3 Homo_sapiens_expr_gold_present.tsv | sort | uniq -c | sort -nr| awk '{print $1 "\t" $2}'  > tissue_count_gold_present.tab

In [ ]:
# gnuplot  -p -e 'plot "tissue_count_gold_present.tab" using 1'

how much coverage do they get?  

![tissue_count](tissue_count.png)  
 
#### for all 
 the first ~ hundred @60000 are well covered and can be left alone,   
the later parts could possibly use some ontological binning,  
if it leads to fewer tissues recorded for more genes.  

![tissue_count](tissue_count_gold.png) 

#### for gold
we loose the top and the right  
and the curve looks like just two main trends  
with no serious discontinuity


![tissue_count](tissue_count_gold_present.png) 

#### for gold present
we loose more the top and the right  
and the curve looks like just two main trends  
with no serious discontinuity

In [16]:
# what is measured
cut -f5 Homo_sapiens_expr_simple.tsv | sort | uniq -c | sort -n

      1 Expression
2893316 absent
5991306 present


In [17]:
# what is measured
cut -f5 Homo_sapiens_expr_gold_present.tsv | sort | uniq -c | sort -n

1909771 present


existance.  (but only among the fraction that are measured),  
for all and gold:  
two thirds are there and one third is not there


Each gene-tissue combination has three states
 - unreported   
 - absent
 - present
 
combining these we can have six combinations 

 - `unreported unreported`
 - `unreported absent`
 - `unreported present`
 - `absent absent`
 - `absent present`
 - `present present`
 
anything and `unreported` should remain `unreported`  
any that agree should strengthen   
any that disagree should weaken  

The first scheme that comes to mind is to let:   
    absent     = -1  
    unreported =  0  
    present    =  1  
then multiply each gene tissue comparison and sum the resulting vector.

note: 
This has the a similar but not as bad effect as my previous scoreing scheme
in [human_protein_atlas_coexpression](https://github.com/TomConlin/human_protein_atlas_coexpression)  
there the best match score approached that of unreported. 
here the 50/50 [agree/disagree] match score aproaches unreported.  

Which at least has a chance of not mattering if significance   
reqires more agreement than disagreement, which seems likely.

They also have a rank.

```
Rank scores of expression calls are normalized across genes, conditions and species. Low score means that the gene is highly expressed in the condition. Max rank score in all species: 4.10e4. Min rank score varies across species.
```
41,000

I am going to ignore rank for now

In [18]:
cut -f7 Homo_sapiens_expr_gold_present.tsv | head

9.77e3
560
6.20e3
2.73e3
9.94e3
1.62e4
1.07e4
2.32e3
5.51e3
1.59e3
cut: write error: Broken pipe


That is far more nuanced than yes|no|maybe

Check up front if our Fanconi genes are represented

In [21]:
cut -f2 Homo_sapiens_expr_gold_present.tsv | tr -d '"' |sort -u > bgee_gene_symbol_gold_present.txt

In [22]:
wc -l < bgee_gene_symbol_gold_present.txt

53481


In [23]:
head FA_1_core_complex.txt

NCBIGene:2175	FANCA
NCBIGene:2187	FANCB
NCBIGene:2176	FANCC
NCBIGene:2178	FANCE
NCBIGene:2188	FANCF
NCBIGene:2189	FANCG
NCBIGene:55120	FANCL
NCBIGene:57697	FANCM
NCBIGene:2177	FANCD2
NCBIGene:55215	FANCI


In [24]:
cut -f2 FA_*.txt | sort > target_gene_symbol.txt

In [25]:
comm -12 bgee_gene_symbol_gold_present.txt target_gene_symbol.txt | wc -l

25


Great! that is 25 out of 27 just checking symbols   
the other two may show up via identifiers   
if any nomenclature is out of date.  

Note 

as I have switched from all, to gold quality, to only gold quality that are present;  
the coverage of the FA set has not decreased
